In [2]:
!pip install google-cloud-documentai
!pip install google-cloud-storage

  Using cached google_cloud_documentai-3.6.0-py3-none-any.whl.metadata (9.7 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio-1.74.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached grpcio_status-1.74.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached google_cloud_documentai-3.6.0-py3-none-any.whl (300 kB)
Using cached google_api_core-2.25.1-py3-none-any.whl (160 kB)
Using cached google_auth-2.40.3-py2.py3-none-any.whl (216 kB)
Using cached proto_plus-1.


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached google_cloud_storage-3.3.1-py3-none-any.whl.metadata (13 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.7.1-cp311-cp311-win_amd64.whl.metadata (2.4 kB)
Using cached google_cloud_storage-3.3.1-py3-none-any.whl (275 kB)
Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl (29 kB)
Using cached google_crc32c-1.7.1-cp311-cp311-win_amd64.whl (33 kB)
Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl (81 kB)



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from google.cloud import documentai
from google.cloud import storage
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# Configure Document AI client
project_id = "sodium-coil-470706-f4"  
location = "us"  # Change if using different region
processor_id = "18d898182b219656"

# Initialize the client
client_options = {"api_endpoint": f"{location}-documentai.googleapis.com"}
client = documentai.DocumentProcessorServiceClient(client_options=client_options)

In [5]:
def process_document(file_path):
    """Process a document using Document AI"""
    # Get the full resource name of processor
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"
    
    # Read the file
    with open(file_path, "rb") as image:
        image_content = image.read()
    
    # Create the document object
    raw_document = documentai.RawDocument(
        content=image_content,
        mime_type="application/pdf"  # Update mime type based on your file
    )
    
    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document
    )
    
    # Process the document
    result = client.process_document(request=request)
    return result.document

In [6]:
def display_results(document):
    """Display the extracted text and other information"""
    print("Full text extracted:")
    print("="*50)
    print(document.text)
    
    print("\nForm fields found:")
    print("="*50)
    for page in document.pages:
        for field in page.form_fields:
            name = get_text(field.field_name, document)
            value = get_text(field.field_value, document)
            print(f"{name}: {value}")
            
def get_text(doc_element, document):
    """Extract text from a document element"""
    text = ""
    for segment in doc_element.text_anchor.text_segments:
        start_index = segment.start_index
        end_index = segment.end_index
        text += document.text[start_index:end_index]
    return text

In [ ]:
# Test the implementation
test_file_path = r"C:\\Users\\braga\\Documents\\College\\Project\\GenAI-exchange\\backend\\uploads\\Chat Application.docx.pdf"  # Update with your test file path

try:
    # Process the document
    processed_doc = process_document(test_file_path)
    
    # Display results
    display_results(processed_doc)
    # Print confidence score
    print("\nDocument confidence score:", processed_doc.text_quality_scores[0].score)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Full text extracted:
UNIFIED MENTOR
YOUR SKILL. SUCCESS & JOURNEY
Project Title
Technologies
inf
Chat Application
HTML, CSS, JavaScript
Project Difficulties level
Hard
Project Description:
In this assignment, you will develop a real-time chat application. The goal is to create a
web-based chat platform where users can join chat rooms, exchange messages in real-time, and
have a smooth and interactive chat experience. You will be using HTML, CSS, and JavaScript for
this project.
Project Requirements:
User Interface:
• Create an intuitive and visually appealing user interface for the chat application
using HTML and CSS.
• Design a chat room interface with a list of available rooms, a message display
area, and an input field for sending messages.
Ensure responsive design to accommodate different screen sizes.
Real-Time Communication:
Implement real-time communication between users using JavaScript and
WebSockets.
• Users should be able to select a chat room to join and exchange messages wi